In [ ]:
# Import dependencies
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset, random_split
from datasets import Dataset
import torch.nn.functional as F
from datasets import load_dataset
import random

In [ ]:
# Dataset and model paths
DATASET_PATH = "scikit-fingerprints/MoleculeNet_Lipophilicity"
MODEL_NAME = "ibm/MoLFormer-XL-both-10pct"
EXTERNAL_DATASET_PATH = "C:/Users/jadej/OneDrive/Desktop/Studies/NNTI 11th Feb/Project/Project_Files/tasks/External-Dataset_for_Task2.csv"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# load the dataset from HuggingFace
dataset = load_dataset(DATASET_PATH)

In [ ]:
# Calculate the maximum length of SMILES strings in the dataset
max_length = max(len(smile) for smile in dataset['train']['SMILES'])
print(f"Max SMILES length: {max_length}")



# define a PyTorch Dataset class for handling SMILES strings and targets

# TODO: your code goes here
# class SMILESDataset(Dataset):


class SMILEDATASET(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.rename(columns={'label': 'Label'})  # Standardize key names

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        return {
            'SMILES': sample['SMILES'],
            'Label': sample['Label']  # Ensure correct key mapping
        }

In [ ]:
# Load external dataset
ext_data = pd.read_csv(EXTERNAL_DATASET_PATH)
# Load the pre-trained model and tokenizer
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# ext_data = ext_data1.head(16)

In [ ]:
ext_data.head()

In [ ]:

# Load pre-trained model and tokenizer

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model.eval()


In [ ]:
def compute_gradients(model, smiles, targets):
    """Compute gradients of loss with respect to model parameters."""
    model.zero_grad()

    # 🔹 Ensure smiles is a string, not a tensor/dict
    if isinstance(smiles, torch.Tensor):
        smiles = smiles.item()  # Convert single tensor to string
    elif isinstance(smiles, list) and isinstance(smiles[0], torch.Tensor):
        smiles = [s.item() for s in smiles]  # Convert list of tensors to strings
    
    inputs = smiles
    # 🔹 Ensure targets are in tensor format
    # targets = torch.tensor([targets], dtype=torch.float32)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    targets = targets.to(device)

    
    # 🔹 Forward pass
    # outputs = model(**inputs)
    outputs = model(input_ids, attention_mask)

    # 🔹 Compute loss
    loss_fn = torch.nn.MSELoss()
    loss = loss_fn(outputs.logits.squeeze(), targets)

    # 🔹 Compute gradients
    loss.backward()
    return [p.grad for p in model.parameters() if p.grad is not None]

In [ ]:
def lissa_approximation(model, test_grad, train_data, iters=100, damping=0.01):
    ihvp = [torch.zeros_like(tg) for tg in test_grad]

    for _ in range(iters):
        random_idx = random.randint(0, len(train_data) - 1)
        train_sample = train_data[random_idx]  # Get one sample

        # Debug print to check data
        # print(f"Sample retrieved: {train_sample}")

        smiles_input = train_sample.get('SMILES', None)
        label_value = train_sample.get('label', None)  # Change from 'Label' to 'label'
        # print("smiles lissa ", smiles_input)
        inputs = tokenizer(train_sample['SMILES'], return_tensors="pt")
        label_value = torch.tensor([train_sample['label']])
        # Safety check: If label or SMILES is missing, raise an error
        if smiles_input is None or label_value is None:
            raise KeyError(f"Missing data in sample: {train_sample}")

        train_grad = compute_gradients(model, inputs, label_value)
        ihvp = [g - damping * ih for g, ih in zip(train_grad, ihvp)]
    
    return ihvp

In [ ]:
def compute_influence_scores(model, external_data, train_data):
    """Compute influence scores for each external data point."""
    influence_scores = []
    for _, row in tqdm(external_data.iterrows(), total=len(external_data)):
        # Tokenize input molecule SMILES string
        inputs = tokenizer(row['SMILES'], return_tensors="pt")
        target = torch.tensor([row['Label']])
        # print("done")
        
        # Compute test gradient
        test_grad = compute_gradients(model, inputs, target)

        # Approximate inverse Hessian-vector product
        ihvp = lissa_approximation(model, test_grad, train_data)

        # Compute influence score (dot product of test gradient and ihvp)
        influence = sum(torch.dot(tg.flatten(), ih.flatten()) for tg, ih in zip(test_grad, ihvp))
        influence_scores.append((row['SMILES'], row['Label'], influence.item()))

    return sorted(influence_scores, key=lambda x: x[1], reverse=True)

In [ ]:
# Load the training split of the Lipophilicity dataset (SMILEDATASET)
SMILEDATASET = load_dataset("scikit-fingerprints/MoleculeNet_Lipophilicity", split="train")

# Assign SMILEDATASET to train_data
train_data = SMILEDATASET

In [ ]:
# Compute influence scores
influence_scores = compute_influence_scores(model, ext_data, train_data)

# Select top-48 influential samples
selected_samples = influence_scores[:48]

In [ ]:
inf_df = pd.DataFrame(influence_scores, columns=["Smiles", "Label", "Influence"])
inf_df.to_csv("Influence.csv", index=False)

In [ ]:
len(influence_scores)

In [ ]:
# selected_samples

In [ ]:
# Load dataset
influence_df = pd.read_csv("Influence.csv")  # Ensure the correct file path

# Select the top 100 SMILES strings and corresponding Labels based on Influence score
top_100 = influence_df.nlargest(100, 'Influence')[["Smiles", "Label"]]  # Extract only required columns

# Convert to lists
top_100_smiles = top_100["Smiles"].tolist()
top_100_labels = top_100["Label"].tolist()



class TopInfluenceDataset(torch.utils.data.Dataset):
    def __init__(self, smiles_list, labels_list, tokenizer, max_length=268):
        self.smiles_list = smiles_list
        self.labels_list = labels_list  # Store labels
        self.tokenizer = tokenizer
        self.max_length = max_length  # Set max length for consistent padding

    def __getitem__(self, idx):
        smiles = self.smiles_list[idx]
        label = torch.tensor(self.labels_list[idx], dtype=torch.float32)  # Convert label to tensor

        encoded = self.tokenizer(
            smiles,
            padding="max_length",  # Ensure all sequences have the same length
            truncation=True,  # Truncate if sequence exceeds max_length
            max_length=self.max_length,
            return_tensors="pt",
        )
        tokenized_data = {key: val.squeeze(0) for key, val in encoded.items()}  # Remove batch dim
        tokenized_data["label"] = label  # Add label to dictionary

        return tokenized_data

    def __len__(self):
        return len(self.smiles_list)


# Create dataset instance
top_100_data_loader = torch.utils.data.DataLoader(
    TopInfluenceDataset(top_100_smiles, top_100_labels, tokenizer),  # Pass only the SMILES list
    batch_size=16,
    shuffle=True,
    # collate_fn=collate_fn  # Ensure proper batching
)

# Define batch size
BATCH_SIZE = 16


# Split train_data into train (80%) and test (20%) sets
train_size = int(0.8 * len(train_data))
test_size = len(train_data) - train_size

train_dataset, test_dataset = random_split(train_data, [train_size, test_size])

# Create DataLoaders for train and test data
train_data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Print dataset sizes
# print(f"Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}, Top 100 dataset: {len(top_100_dataset)}")

In [ ]:
test_dataset_prepared

In [ ]:
# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
num_epochs = 5  # Adjust if needed
best_loss = float('inf')  # Initialize best loss to a large value

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    progress_bar = tqdm(top_100_data_loader, desc=f"Fine-Tuning Regression (MLM) Epoch {epoch+1}/{num_epochs}", leave=False)
    # print(top_100_data_loader)
    for batch in top_100_data_loader:
        # print(batch)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        # print(input_id)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
       
       # Extract the predicted values corresponding to the true labels
        outputs_reshaped = outputs.logits[:, 0] # Flatten logits
        loss = criterion(outputs_reshaped, labels.float())  # Ensure labels are float for MSELoss

        # loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        progress_bar.set_postfix({"Training Loss": loss.item()})

    avg_epoch_loss = epoch_loss / len(top_100_data_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_epoch_loss:.4f}")

In [ ]:
test_data_loader

In [ ]:
# Define Loss and Optimizer for eval
    # === EVALUATION PHASE ===
model.eval()
eval_loss = 0.0
with torch.no_grad():  # Disable gradient calculations
    for batch in test_data_loader:  # Use test data
        # print(batch)
        # input_ids = batch['input_ids'].to(device)
        # attention_mask = batch['attention_mask'].to(device)
        # labels = batch['label'].to(device)

        smiles_list = batch["SMILES"]  # Extract SMILES strings
        labels = batch["label"].to(torch.float).to(device)  # Convert labels to tensors
        
        encoding = tokenizer(
            smiles_list,
            padding="max_length",
            truncation=True,
            max_length=128,  # Adjust max_length as needed
            return_tensors="pt",
        )
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)
        
        outputs = model(input_ids, attention_mask)
        outputs_reshaped = outputs.logits[:, 0] # Flatten logits
        loss = criterion(outputs_reshaped, labels)
        eval_loss += loss.item()

avg_eval_loss = eval_loss / len(test_data_loader)

# Track Best Model
if avg_eval_loss < best_loss:
    best_loss = avg_eval_loss
    torch.save(model.state_dict(), "best_model.pth")  # Save best model

print(f"Epoch {epoch+1}/{num_epochs} | Eval Loss: {avg_eval_loss:.4f}")

print(f"Best Evaluation Loss: {best_loss:.4f}")

In [ ]:

print(f"Epoch {epoch+1}/{num_epochs} | Eval Loss: {avg_eval_loss:.4f}")

print(f"Best Evaluation Loss: {best_loss:.4f}")